In [8]:
!pip install -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.8.5
    Uninstalling google-generativeai-0.8.5:
      Successfully uninstalled google-generativeai-0.8.5


In [1]:
import os
import json
from google.colab import userdata
import google.generativeai as genai

def configure():
    secret_name = "KAY_API_KEY"
    api_key = userdata.get(secret_name)

    if not api_key:
        raise ValueError(f"Secret '{secret_name}' not found.")

    genai.configure(api_key=api_key)

    # Updated for 2026: Use gemini-2.5-flash for the best balance of speed and power
    # or use 'gemini-3-flash-preview' for the absolute latest.
    model_name = 'gemini-2.5-flash'

    # We define the model here, but we don't attach tools yet
    # so that the Planner doesn't try to use Google Search for logic.
    return genai.GenerativeModel(model_name)

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [2]:
def planner_agent(model, topic: str) -> list[str]:
    print("Planner Agent: Creating a research plan...")

    # FIX 2: Request structured JSON to avoid manual string cleaning errors
    prompt = f"""
    You are an expert research planner. Break down the topic: "{topic}"
    into 3 specific, answerable questions.
    Return the result as a raw JSON list of strings.
    Example: ["question 1", "question 2", "question 3"]
    """

    try:
        # Using generation_config to ensure we get JSON back
        response = model.generate_content(
            prompt,
            generation_config={"response_mime_type": "application/json"}
        )

        plan = json.loads(response.text)
        print("Plan created:")
        for i, q in enumerate(plan, 1):
            print(f"   {i}. {q}")
        return plan
    except Exception as e:
        print(f"Error in Planner Agent: {e}")
        # Fallback if JSON fails
        return [f"General overview of {topic}"]

In [5]:
def search_agent(model, question: str) -> str:
    print(f"Search Agent: Researching question: '{question}'...")
    try:
        # 2026 Fix: Use the new tool name 'google_search'
        # and use the simple dictionary format for maximum compatibility.
        search_tool = {"google_search": {}}

        prompt = f"Provide a detailed answer to the following question using a web search: {question}"

        # We pass the tool directly in the list
        response = model.generate_content(
            prompt,
            tools=[search_tool]
        )

        if response.text:
            print("   - Information found.")
            return response.text
        return "No specific information found."

    except Exception as e:
        print(f"Error in Search Agent: {e}")
        return ""


def synthesizer_agent(model, topic: str, research_results: list) -> str:
    print("Synthesizer Agent: Writing the final report...")

    research_notes = ""
    for question, data in research_results:
        research_notes += f"### Question: {question}\n### Research Data:\n{data}\n\n---\n\n"

    prompt = f"""
    You are an expert research analyst. Synthesize these notes into a report on: "{topic}".
    Include an introduction, key findings, and a conclusion.

    ## Research Notes ##
    {research_notes}
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error in Synthesizer Agent: {e}")
        return "Error: Could not generate report."

In [6]:
def main():
    try:
        model = configure()
    except Exception as e: # Catching general config errors
        print(f"Configuration failed: {e}")
        return

    print("\nHello! I am your AI Research Assistant.")
    topic = input("What topic would you like me to research today? ").strip()

    if not topic:
        print("A topic is required to begin research. Exiting.")
        return

    print(f"\n[Step 1] Creating research plan for: '{topic}'")
    research_plan = planner_agent(model, topic)

    # Check if the plan is actually a list (robustness check)
    if not isinstance(research_plan, list) or len(research_plan) == 0:
        print("Could not create a valid research plan. Exiting.")
        return

    research_results = []
    print(f"\n[Step 2] Researching {len(research_plan)} key areas...")

    for i, question in enumerate(research_plan, 1):
        print(f"   Searching ({i}/{len(research_plan)}): {question}")
        research_data = search_agent(model, question)
        if research_data:
            research_results.append((question, research_data))

    if not research_results:
        print("Could not find any information during research. Exiting.")
        return

    print("\n[Step 3] Synthesizing findings into a report...")
    final_report = synthesizer_agent(model, topic, research_results)

    print("\n" + "="*40)
    print(f"FINAL RESEARCH REPORT: {topic.upper()}")
    print("="*40)
    print(final_report)
    print("="*40 + "\n")

if __name__ == "__main__":
    main()


Hello! I am your AI Research Assistant.
What topic would you like me to research today? weather

[Step 1] Creating research plan for: 'weather'
Planner Agent: Creating a research plan...
Plan created:
   1. How do variations in atmospheric pressure, temperature, and humidity interact to create different types of precipitation (e.g., rain, snow, hail)?
   2. What are the primary short-term and long-term impacts of extreme weather events (e.g., heatwaves, floods, droughts) on human health and infrastructure?
   3. What technological advancements and data sources (e.g., satellite imagery, Doppler radar, numerical weather models) have most significantly improved the accuracy of 7-day weather forecasts?

[Step 2] Researching 3 key areas...
   Searching (1/3): How do variations in atmospheric pressure, temperature, and humidity interact to create different types of precipitation (e.g., rain, snow, hail)?
Search Agent: Researching question: 'How do variations in atmospheric pressure, tempera